In [25]:
from biocrnpyler import *
 # Will likely need to import other things instead of this 


A note on the naming convention: 
- Species are identified as shared between mixtures if they have the same name (the string)
- They may have different components or compartments, but reactions will be made with the species names (?)


Other questions/ concerns:
- I am concerned abotu adding the compartment to mixture copies. Also, the mixture copies just take up so much space. 
- Mixture class can benefit from compartment attribute 
- Do i need to return the compartment as well? 
- Do i need to explicitly add compartments?

In [32]:
class MultiMixtureGraph(object):
    def __init__(self, name="",
                  mixtures=None, 
                  parameters=None, 
                  parameter_file=None,
                  **kwargs):
        self.name = name
        self.mixture_graph = {}
        self.compartment_map = {} # This maps mixtures to their compartments 
        
        
        if mixtures is None:
            self.mixtures = []
        else:
            self.add_mixture(mixtures)
            
    def add_mixture(mixture, compartment = None):
        if isinstance(mixture, Mixture): 
            
            # create compartment 
            if compartment == None:
                compartment = Compartment(name = mixture.name)
            
            mixture_copy = copy.deepcopy(mixture)
      
            # add compartment to compartment map 
            compartment_map[mixture_copy, compartment]
            # TODO: set MultiMixtureGraph (like done in component). This would require changing the Mixture class 
            
            self.mixture_graph[mixture_copy] = []
            self.mixtures.append(mixture_copy)
            return mixture_copy
        
        
        elif isinstance(mixture, List):
            return self.add_mixtures(mixture)
        else:
            raise ValueError("You did not input a Mixture or list of Mixtures")
            
    def add_mixtures(mixtures, compartments):
        if isinstance(mixtures, Mixture) and isinstance(compartments, Compartment) :
            return self.add_mixture(mixtures)
        elif isinstance(mixtures, List) and isinstance(compartments, List):
            mixtures_list = []
            for i in len(mixtures):
                mixtures_list.append(self.add_mixture(mixtures[i], compartments[i]))
            return mixtures_list 
        else:
            raise ValueError("You did not input a Mixture/ Compartment or list of Mixtures / list of Compartments")
                
                
    def add_transport(mixture_1, mixture_2, transport_mixture):
        
        
        # Checking that each adjacent mixture actually shares species 
        # added_species was from the mixture class. perhaps we can add an "add species" method to Mixture class
        mixture_1_species = mixture_1.added_species 
        mixture_2_species = mixture_2.added_species
        transport_mixture_species = transport_mixture.added_species
        
        # add more specific naming in error and change in tests 
        if not bool((set(mixture_1_species)).intersection(set(transport_mixture_species))):
            raise ValueError("There are no shared species between mixture_1 and transport_mixture")
        if not bool((set(mixture_2_species)).intersection(set(transport_mixture_species))):
            raise ValueError("There are no shared species between mixture_2 and transport_mixture")
        
        # this is really just for the unit tests, not sure how necessary it is 
      
        
        # when would this ever be applicable?
        if not mixture_1 in self.mixture_graph:
            mixture_1 = self.add_mixture(mixture_1)
        if not mixture_2 in self.mixture_graph:
            mixture_2= self.add_mixture(mixture_2)
        if not transport_mixture in self.mixture_graph:
            transport_mixture = self.add_mixture(transport_mixture)
            
        # Here, we want to check that there is at least 1 shared species 
        # It will be good for the renaming process 
        
   
        self.mixture_graph[mixture_1].append(transport_mixture)
        self.mixture_graph[transport_mixture].append(mixture_2)
        self.mixture_graph[mixture_2].append(transport_mixture)
        self.mixture_graph[transport_mixture].append(mixture_1)
        
        # Here, we add the compartment logic. should internal and external be with the membranes?
        
        compartment_map[mixture_1].external = compartment_map[transport_mixture]
        compartment_map[transport_mixture].external = compartment_map[mixture_2]
        compartment_map[transport_mixture].internal = compartment_map[mixture_1]
        compartment_map[mixture_2].internal = compartment_map[transport_mixture]
        
        return [mixture_1, mixture_2, transport_mixture] 

    def duplicate_structure(mixture, duplicate_external):
        # This assumes that mixture is something that is in the graph 
        # Duplicate_external is a boolean to decide if we duplicate the external of an object as well 
        
        if mixture not in self.mixtures:
            self.add_mixture(mixture)
        else:
            mixture_copy = copy.deepcopy(mixture) 
            compartment_copy =copy.deepcopy(compartment_map[mixture])
            if not duplicate_external:
                compartment_copy.compartment_dict['external'] = []

            # it will again deep copy in the add_mixture method
            copied_mixture = self.add_mixture(mixture_copy, compartment_copy)
            
            # recreating graph connections, conditionally including external if specified 
            for item in self.mixture_graph[mixture]:
                if(compartment_map[item] not in compartment_map[mixture].external):
                    cpy = copy.deepcopy(item)
                    self.mixture_graph[item].append(copied_mixture)
                    self.mixture_graph[copied_mixture].append(item)
                elif (duplicate_external):
                    cpy = copy.deepcopy(item)
                    self.mixture_graph[item].append(copied_mixture)
                    self.mixture_graph[copied_mixture].append(item)
            this.add_transport()
        
    def remove_mixture(mixture):
        # TODO 
        pass
    def get_mixtures():
        return self.mixture_graph.keys()
    def get_graph():
        return self.mixture_graph
    def get_compartment_map():
        return self.compartment_map
    def print_graph():
        pass 
    def compile_crn(self, **kwargs) -> ChemicalReactionNetwork:
        crn_species = []
        crn_reactions = []
        for mixture in self.mixture_graph.keys():
            
            temp = mixture.compile_crn(kwargs) 
            specs = temp.species
            
            # adding the compartment to each species
            for species in specs:
                species.compartment(compartment_map[mixture])
                
            crn_species.append(specs) 
            crn_reactions.append(temp.reactions)
        self.crn = ChemicalReactionNetwork(crn_species, crn_reactions)
        
        


Notes for self:
- add value errors for unit testing for add mixture 

Checking that intersection of two sets checks strings structurally, not for identity. 

In [20]:
l1 = {"species1", "species2", "species3"}
l2 = {"species1", "species4", "species5"}

if  not bool(set(l1).intersection(set(l2))):
    print("mpt")
else:
    print("full")

full


With the deep copy, a new mixture object will be created, so it is unlikely that someone will accidentally pass it in (for the ValueError thrown in add_mixture). This also might make adding things to add_transport interesting. 

Perhaps we don't need to do deep copy, and if someone wants to reuse something, they could deepcopy on the user side.


## Unit Tests

In [28]:
from unittest import TestCase
from biocrnpyler import Mixture

class TestMultiMixtureGraph(TestCase): 
    
    def test_add_mixture(self):
        mmg = MultiMixtureGraph() 
        mixture1 = Mixture('test_mixture1')
        mixture1_cpy = mmg.add_mixture(mixture1)

        # Checking that the internal list of mixtures is omly the added mixture 
        self.assertEqual([mixture_cpy], mmg.mixtures)
        # Checking that the mixture has been added to the dictionary correctly 
        self.assertEqual(False, mixture1 in mmg.mixture_graph)
        self.assertEqual(True, mixture1_cpy in mmg.mixture_graph)
        # Checking that the string representation of the mixture and its copy are the same 
        self.assertTrue(mixture1_cpy.name == mixture.name)
        
        # Testing behavior with multiple mixtures
        
        mixture2 = Mixture('test_mixture2')
        mixture3 = Mixture('test_mixture3')
        mixture_list = [mixture2, mixture3]
        
        mixture_copy_list = mmg.add_mixture(mixture_list)
        self.assertEqual([mixture1_cpy, mixture_copy_list[0], mixture_copy_list[1]], mmg.mixtures)
        self.assertEqual(False, mixture2 in mmg.mixture_graph and mixture2 in mmg.mixture_graph)
        self.assertEqual(True, mixture_copy_list[0] in mmg.mixture_graph and mixture_copy_list[1] in mmg.mixture_graph)
        
            
    def test_add_mixtures(self):
        mmg = MultiMixtureGraph() 
        mixture1 = Mixture('test_mixture1')
        mixture1_cpy = mmg.add_mixtures(mixture1)
        
        # Testing behavior with a single mixture 

        # Checking that the internal list of mixtures is only the added mixture 
        self.assertEqual([mixture_cpy], mmg.mixtures)
        # Checking that the mixture has been added to the dictionary correctly 
        self.assertEqual(False, mixture1 in mmg.mixture_graph)
        self.assertEqual(True, mixture1_cpy in mmg.mixture_graph)
        # Checking that the string representation of the mixture and its copy are the same 
        self.assertTrue(mixture1_cpy.name == mixture.name)
        
        # Testing behavior with multiple mixtures
        
        mixture2 = Mixture('test_mixture2')
        mixture3 = Mixture('test_mixture3')
        mixture_list = [mixture2, mixture3]
        
        mixture_copy_list = mmg.add_mixture(mixture_list)
        self.assertEqual([mixture1_cpy, mixture_copy_list[0], mixture_copy_list[1]], mmg.mixtures)
        self.assertEqual(False, mixture2 in mmg.mixture_graph and mixture2 in mmg.mixture_graph )
        self.assertEqual(True, mixture_copy_list[0] in mmg.mixture_graph and mixture_copy_list[1] in mmg.mixture_graph)
      
    def test_add_transport(self):
        species1 = Species('test_species1')
        species2 = Species('test_species2')
        species3 = Species('test_species3')
        
        mmg = MultiMixtureGraph() 
        mixture1 = Mixture('test_mixture1')
        transport_mixture = Mixture('test_transport_mixture')
        mixture2 = Mixture('test_mixture2')
        
        mixture1.add_species(species1)
        transport_mixture.add_species(species2)
        mixture2.add_species(species3)
        
        with self.ValueError("There are no shared species between mixture_1 and transport_mixture"):
            mmg.add_transport(mixture1, transport_mixture, mixture_2)
            
        transport_mixture.add_species(species1)
        
        with self.ValueError("There are no shared species between mixture_2 and transport_mixture"):
            mmg.add_transport(mixture1, transport_mixture, mixture_2)
            
        transport_mixture.add_species(species3)
        
        
            
            
        # test adding with no mixtures in there
        [m1, m2, transp_m] = mmg.add_transport(mixture1, transport_mixture, mixture_2)
        self.assertEqual(True, m1 in mmg.mixture_graph)
        self.assertEqual(True, m2 in mmg.mixture_graph)
        self.assertEqual(True, transp_m in mmg.mixture_graph)
        
        
#         self.mixture_graph[mixture_1].append(membrane_mixture)
#         self.mixture_graph[transport_mixture].append(mixture_2)
#         self.mixture_graph[mixture_2].append(membrane_mixture)
#         self.mixture_graph[transport_mixture].append(mixture_1)
        
        self.assertTrue(mmg.mixture_graph[m1].contains(transp_m))
        self.assertTrue(mmg.mixture_graph[transp_m].contains(m1))
        self.assertTrue(mmg.mixture_graph[transport_mixture].contains(m2))
        self.assertTrue(mmg.mixture_graph[m2].contains(transp_m))
        
        
        # Testing with mixtures already in there 
        mixture3 = Mixture('test_mixture3')
        mixture4 = Mixture('test_mixture4')
        transport_mixture2 = Mixture('test_transport_mixture2')
        
        mixture3.add_species(species1)
        mixture4.add_species(species3)
        transport_mixture2.add_species(species2)
        
        mixtures_list = mmg.add_mixtures([mixture3, mixture4, transport_mixture2])
        mix3 = mixtures_list[0]
        mix4 = mixtures_list[1]
        transp_mix2 = mixtures_list[2]
        
        
        with self.ValueError("There are no shared species between mixture_1 and transport_mixture"):
            mmg.add_transport(mix3, transp_mix2, mix4)
        
        transport_mixture2.add_species(species3)
        with self.ValueError("There are no shared species between mixture_2 and transport_mixture"):
            mmg.add_transport(mix3, transp_mix2, mix4)
    
        transport_mixture2.add_species(species1)
        
        self.assertTrue(mmg.mixture_graph[mix3].contains(membrane_mixture))
        self.assertTrue(mmg.mixture_graph[transp_mix2].contains(mix3))
        self.assertTrue(mmg.mixture_graph[transp_mix2].contains(mix4))
        self.assertTrue(mmg.mixture_graph[mix4].contains(transp_mix2))
        
        
    def test_remove_mixture(self): 
        pass
    
    def test_get_mixtures(self):
        mmg = MultiMixtureGraph() 
        mixture1 = Mixture('test_mixture1')
        mixture2 = Mixture('test_mixture2')
        
        mixtures_list = mmg.add_mixtures[mixture1, mixture2]
        
        assertEqual(mmg.get_mixtures,[mixtures_list[0], mixtures_list[1]])
    
    def test_get_graph(self):
        mmg = MultiMixtureGraph() 
        mixture1 = Mixture('test_mixture1')
        mixture2 = Mixture('test_mixture2')
        g = {}
        g[mixture1] = []
        g[mixture2] = []
        
        mixtures_list = mmg.add_mixtures[mixture1, mixture2]
        assertEqual(mmg.get_graph, g)
    
    def test_print_graph(self):
        pass 
    
    def test_compile_crn(self):
        pass
    # TODO 

Naming convention of species
- name is a string
- have a compartment they are in 

The species check:
- check for name structural equality, not whole equality
- then check the compartment thing 

The compartment check:
- need to check that they are each others internals and externals after checking species 

# MultiMixtureGraph without Compartments 

In [ ]:
class MultiMixtureGraph(object):
    def __init__(self, name="",
                  mixtures=None, 
                  parameters=None, 
                  parameter_file=None,
                  **kwargs):
        self.name = name
        self.mixture_graph = {}
        if mixtures is None:
            self.mixtures = []
        else:
            self.add_mixture(mixtures)
            
            
    def add_mixture(mixture):
        if isinstance(mixture, Mixture): 
#             if mixture in self.mixture_graph: # look into how dictionary evaluates keys "contains"-- make sure it works in unit tests
#                 raise ValueError("This mixture has already been added to the mixture_graph")\

            mixture_copy = copy.deepcopy(mixture)
            # TODO: set MultiMixtureGraph (like done in component). This would require changing the Mixture class 
            self.mixture_graph[mixture_copy] = []
            self.mixtures.append(mixture_copy)
            return mixture_copy
        elif isinstance(mixture, List):
            return self.add_mixtures(mixture)
        else:
            raise ValueError("You did not input a Mixture or list of Mixtures")
            
    def add_mixtures(mixtures):
        if isinstance(mixtures, Mixture):
            return self.add_mixture(mixtures)
        elif isinstance(mixtures, List):
            mixtures_list = []
            for mixture in mixtures:
                mixtures_list.append(self.add_mixture(mixture))
            return mixtures_list 
        else:
            raise ValueError("You did not input a Mixture or list of Mixtures")
    
                
                
    def add_transport(mixture_1, mixture_2, transport_mixture):
        
        
        # Checking that each adjacent mixture actually shares species 
        # added_species was from the mixture class. perhaps we can add an "add species" method to Mixture class
        mixture_1_species = mixture_1.added_species 
        mixture_2_species = mixture_2.added_species
        transport_mixture_species = transport_mixture.added_species
        
        # add more specific naming in error and change in tests 
        if not bool((set(mixture_1_species)).intersection(set(transport_mixture_species))):
            raise ValueError("There are no shared species between mixture_1 and transport_mixture")
        if not bool((set(mixture_2_species)).intersection(set(transport_mixture_species))):
            raise ValueError("There are no shared species between mixture_2 and transport_mixture")
        
        # this is really just for the unit tests, not sure how necessary it is 
      
        
        # when would this ever be applicable?
        if not mixture_1 in self.mixture_graph:
            mixture_1 = self.add_mixture(mixture_1)
        if not mixture_2 in self.mixture_graph:
            mixture_2= self.add_mixture(mixture_2)
        if not transport_mixture in self.mixture_graph:
            transport_mixture = self.add_mixture(transport_mixture)
            
        # Here, we want to check that there is at least 1 shared species 
        # It will be good for the renaming process 
        
   
        self.mixture_graph[mixture_1].append(membrane_mixture)
        self.mixture_graph[transport_mixture].append(mixture_2)
        self.mixture_graph[mixture_2].append(membrane_mixture)
        self.mixture_graph[transport_mixture].append(mixture_1)
        
        return [mixture_1, mixture_2, transport_mixture] 
    
        
    def remove_mixture(mixture):
        # TODO 
        pass
    def get_mixtures():
        return self.mixture_graph.keys()
    def get_graph():
        return self.mixture_graph
    def print_graph():
        pass 
    def compile_crn(self, recursion_depth = None, initial_concentration_dict = None, return_enumerated_components = False,
        initial_concentrations_at_end = False, copy_objects = True, add_reaction_species = True) -> ChemicalReactionNetwork:
        crn_species = []
        crn_reactions = []
        for mixture in self.mixture_graph.keys():
            temp = mixture.compile_crn()
            crn_species.append(temp.species)
            crn_reactions.append(temp.reactions)
        self.crn = ChemicalReactionNetwork([set(crn_species)], [set(crn_reactions)])
# user will want to input two main mixtures, with one method of transportation. unlikely 
# that it'll be more than 1 method of transport 
# but let's keep it how it is. dictionary would be all mixtures 